
## FINANCIAL DATA
MODULE 4 | LESSON 1


---


# **OPTION PAYOFFS AND STRATEGIES**

|  |  |
|:---|:---|
|**Reading Time** |  30 minutes |
|**Prior Knowledge** | Calls, Puts, Strikes, Volatility, Option payoffs  |
|**Keywords** |Break-even levels, Straddle, Strangle, Iron condor   |


---


*In the previous module, we examined stocks and cryptocurrencies.  In this module, we will examine the use of Python to compute and visualize option payoffs and prices, from single options to multi-option strategies.*

In [ ]:
import math

from optionprice import Option

## 1. Options
In this lesson, we'll use a Python package called `optionprice`.  This has a function called Option. As you recall from Module 4 of Financial Markets, there are two fields that describe the type of option:
1. The type of option: call or put. In the Option function, this is specified by the argument kind.
2. The style of exercise: European or American. In the Option function, this is specified with the European argument set to True or False.

There are also six numerical values that allow us to value the option:
1. Strike Level: k
2. Stock Price: s0
3. Time to expiration (here, this is given in calendar days): t
4. Volatility: sigma (which is annualized)
5. Risk-Free rate: r (which is annualized)
6. Dividend-Yield: dv (which is annualized)
Let's import the package and create an ATM European call option. The function Option helps us to do that. Notice that we have to specify all eight of these fields as arguments to the Option function.

In [ ]:
myCall = Option(
    european=True, kind="call", s0=100, k=100, t=30, sigma=0.20, r=0.05, dv=0
)
print(myCall)

In [ ]:
type(myCall)

Similarly, to create a put, we merely need to change the type to put.

In [ ]:
myPut = Option(european=True, kind="put", s0=100, k=100, t=30, sigma=0.20, r=0.05, dv=0)
print(myPut)

If we want a single attribute, we can just query it from the option.

In [ ]:
myCall.european

In [ ]:
myCall.k

In [ ]:
myCall.kind  # 1 for calls, and 2 for puts

In [ ]:
myCall.s0

In [ ]:
myCall.sigma

In [ ]:
myCall.r

## 2. Getting the Option's Price

As you first learn options, be sure to distinguish the price from the payoff.  
The option's payoff is known precisely at expiration. Indeed, the payoff only applies at an option's expiration.
The option's price applies at every time from inception up until the payoff.
Clearly, pricing is a much more difficult undertaking than figuring out the payoff.

Let's learn to use pricing functions that are part of this Python module.
It is one thing to use a function to get an option's price.  
It's another to understand what that function does.
Keep in mind that there is an entire course, Derivative Pricing, that will provide the mathematics and intuition behind the pricing of these securities.
For now, let's just outline that there are two methods for pricing an option:
1. Analytic
2. Numeric

An analytic method uses mathematics, such as stochastic calculus, Fourier transforms, stochastic dominance, and other analytical methods, to develop a pricing equation.
A numeric method uses numerical or computational techniques, such as discretization (binomial trees, finite difference methods, finite element methods) or randomization (Monte Carlo sampling, agent-based models) to develop a numerical method through which the option price can be found.

Let's run one example of an analytical model called Black Scholes.
Let's run two examples of numerical models: binomial tree and Monte Carlo.

In the following code excerpt, we price our call three different ways:
1. Analytically with the Black Scholes model.
2. Numerically with a binomial tree.
3. Numerically with a Monte Carlo simulation.

In [ ]:
callBS = round(myCall.getPrice(), 4)
callBT = round(myCall.getPrice(method="BT", iteration=5000), 4)
callMC = round(myCall.getPrice(method="MC", iteration=500000), 4)
[callBS, callBT, callMC]

As we can see, the prices are fairly close. This is to be expected. If we use the methods properly, the prices should not vary depending on which method we use.  Keep in mind that the numeric methods (e.g., Monte Carlo simulation) require a sufficiently high number of iterations so that they can converge to the proper solutions.
Now that we've priced the call option, let's price the put using these three methods.

In [ ]:
putBS = round(myPut.getPrice(), 4)
putBT = round(myPut.getPrice(method="BT", iteration=5000), 4)
putMC = round(myPut.getPrice(method="MC", iteration=500000), 4)
[putBS, putBT, putMC]

Again, we have good agreement among the prices. Intentionally, we did not run a high number of iterations for the Monte Carlo method to illustrate how the results can be slightly off.

Whenever we compute option prices, we should get in the habit of checking our prices using put-call parity. Module 4 of Financial Markets showed us that:
$$call - put = S - Ke^{-(rt)}$$  
Let's check this for ourselves.  
Going forward, we'll use the option prices from the analytical form.
First, let's calculate the left-hand side of the put-call parity equation.

In [ ]:
c = callBS
p = putBS
round(c - p, 6)

In [ ]:
S = 100
K = 100
r = 0.05
t = 30 / 365
round(S - K * math.exp(-r * t), 6)

The difference is small, likely due to rounding and not an actual arbitrage.  Ensuring put-call parity holds means that our calculations have a consistency. Whenever you are asked to compute option prices, always check put-call parity. It avoids your making a careless mistake.
For any questions you see on CRTs or GWPs, be sure to apply put-call parity.
As you can see, it is computationally very easy.
Conceptually, it is quite intriguing.
It says that at any given time, a relationship among three different markets exists.
The 3 markets are:
1. options market: calls and puts
2. stock market: underlying stock
3. bond market: a risk-free bond whose term is the option's expiration 

Now that we've seen some option payoffs and prices, let's continue our applied treatment of options by examining some of the strategies.

## 3. Option Strategies

We've been using an option package called `optionprice` for creating options and examining their prices.  In fact, there are several Python packages available for handling options.  We now turn to one called `opstrat` that is useful for illustrating option strategies.

In [ ]:
import opstrat as op

Let's graph the call option's payoff.  
Let's assume that we paid nothing for the option to see its payoff.
The function single_plotter will take the following arguments:
1. spot: current stock price
2. strike: strike level
3. op_type: option type, either "c" for call or "p" for put
4. tr_type: side of trade, either "b" for buying/long or "s" for selling/short
5. op_pr: option price


In [ ]:
op.single_plotter(spot=100, strike=100, op_type="c", tr_type="b", op_pr=0)

Remember, that the option's payoff you see in books tends to neglect the premium you paid. Once you buy an option, you can not lose any additional money. However, long option positions can lose money if the options expires OTM.
Since we bought the call, we paid the $2.49 premium. Let's redraw the payoff taking the premimum into account.


In [ ]:
op.single_plotter(spot=100, strike=100, op_type="c", tr_type="b", op_pr=callBS)

Clearly, the option is unprofitable not only if the stock price is below the strike level but also if the stock price is less than the sum of the strike and premium. So even if the option ends up ITM, we have to cover the cost of the premium to have a positive P&L.  

In fact, we can compute a break-even stock level for a call option.
The **break-even stock level** is the price at which the option holder has a net payoff of 0.
Call Break-Even Stock Level = Strike + Option Premium
In our example, this is 
Call Break-Even Stock Level  = 100 + 2.49
                             = \$102.49 
The break-even is where the stock payoff crosses the x-axis.

Let's see the seller's payoff.

In [ ]:
op.single_plotter(spot=100, strike=100, op_type="c", tr_type="s", op_pr=callBS)

Likewise, the seller's payoff for a call can be positive so long as the stock price exceeds the strike less the premium. To remind us that options are a zero-sum game, let's examine the two payoff diagrams together.


In [ ]:
op.single_plotter(spot=100, strike=100, op_type="c", tr_type="b", op_pr=callBS)
op.single_plotter(spot=100, strike=100, op_type="c", tr_type="s", op_pr=callBS)

## 4. Zero Sum Games

Options are a zero sum game.  The buyer of the call has a payoff that is the mirror image of the seller's payoff.
Everywhere the long position is profitable (green), the seller is unprofitable (red).  
Everywhere the long position is unprofitable (red), the seller is profitable (green).

Notice the non-linearity in each graph.  
The strike is where the change in slope occurs.  
To one side of the strike, the payoff is a flat line.
To the other side of the strike, the payoff is 45 degrees from the x-axis.
For long positions, the slope is upwards.
For short positions, the slope is downwards.
Note that long calls have unlimited upside; therefore, short calls have unlimited downside.
As this graph reminds us, it is prudent to hedge all short calls.

For completion, let's compute the break-even for puts and then graph long and short puts.

Put Break Even Stock Level = Option Strike - Premium
                           = 100 - 2.08
                           = 97.92
The put break-even stock level is where the option payoff crosses the x-axis.

In [ ]:
op.single_plotter(spot=100, strike=100, op_type="p", tr_type="b", op_pr=putBS)
op.single_plotter(spot=100, strike=100, op_type="p", tr_type="s", op_pr=putBS)

We see the same story: 
The long and short payoffs are mirror images of each other, reflecting the zero-sum game of options.
At any given stock price, one side has a profit (shown in green) and the other side has a loss (shown in red).
Each graph is non-linear.  
The slopes are still 45 degrees; however, for the long position, the slope is negative,
and for the short position, the slope is positive.
Note: Even if the option is ITM, the buyer still needs to cover the cost of the premium for the option to be profitable. Also, it is relatively easy to lose 100%.
 

## 5. Option Combinations

Another strategy involves trading volatility.  
Suppose we are unsure of the direction but believe that volatility will increase.
We can buy a call and a put at the same strike. This is an option straddle.

Remember, you don't have to get the direction of the underlying right.


In [ ]:
myCall = {"op_type": "c", "strike": 100, "tr_type": "b", "op_pr": 2.49}
myPut = {"op_type": "p", "strike": 100, "tr_type": "b", "op_pr": 2.09}
op_list = [myCall, myPut]
op.multi_plotter(spot=100, spot_range=10, op_list=op_list)

Notice the position is unprofitable unless the price moves about \$4.58 in either direction.
Option strategies that involve buying two options mean that the position has to cover the cost of two premiums.

In [ ]:
myCall2 = Option(
    european=True, kind="call", s0=100, k=110, t=30, sigma=0.20, r=0.05, dv=0
)
myPut2 = Option(european=True, kind="put", s0=100, k=90, t=30, sigma=0.20, r=0.05, dv=0)
callBS2 = round(myCall2.getPrice(), 4)
putBS2 = round(myPut2.getPrice(), 4)
print([callBS2, putBS2])

Now we construct an option strategy with four options.

In [ ]:
myCall = {"op_type": "c", "strike": 100, "tr_type": "b", "op_pr": callBS}
myPut = {"op_type": "p", "strike": 100, "tr_type": "b", "op_pr": putBS}
myCall2 = {"op_type": "c", "strike": 110, "tr_type": "s", "op_pr": callBS2}
myPut2 = {"op_type": "p", "strike": 90, "tr_type": "s", "op_pr": putBS2}
op_list = [myCall, myPut, myCall2, myPut2]
op.multi_plotter(spot=100, spot_range=20, op_list=op_list)

## 6. Comparing Volatility Strategies

Plan A:  Buy a straddle.   Cost: \$4.59.

Plan B:  Buy an iron butterfly. \$(4.59 - 0.20) 
                                 =\$4.39

The iron butterfly involves a long straddle (buying a call and put at the same strike) and also a short straddle (selling an OTM put at a low strike and selling an OTM call at a high strike).
The iron butterfly gives up a potential upside for a rebate of the option premium.
Individually, we have 
1 short put at 90
1 long call at 100
1 long put at 100
1 short call at 110

The long iron butterfly trader is conservatively long volatility: even if volatility increases significantly, the long iron butterfly's P&L is capped at about \$5.  That's because the long call's gain past \\$110 level is offset by the loss in the short call at strike 110.

The long straddle trader is aggressively long volatility: the more volatility, the more their P&L.

For example, if the final stock price is 115, then 
short put(90)   =  0
long put(100)   =  0
long call(100)  = 15
short call(110) = -5
Net Profit =     \$10

Indeed, \\$10 is the maximum revenue. When the \\$4.39 cost is taken into consideration, the maximum P&L is 
\$10 - \$4.39 = \$5.61



## 7. Saving
Graphs are good for visualizing. Sometimes, the intended consumer of the graph is just the programmer/analyst, as they confirm their findings. Other times, some graphs are meant for the traders, portfolio managers, or risk managers.

Python makes it easy to save plots. Here's a simple function to save the plot as a .jpeg file.


In [ ]:
op.multi_plotter(
    spot=100, spot_range=20, op_list=op_list, save=True, file="myOptionStrategy.jpeg"
)

## 8. Conclusion

In this lesson, we used two Python packages, `optionprice` and `opstrat`, to create and visualize options and their strategies. We examined the two methods of pricing options: analytical and numerical.  We examined option strategies that use single options and multiple options.  We related the level of bullishness or bearishness to the type of option strategy using combinations of calls and puts at different strikes.  In the next module, we examine the sensitivities of options to their underlying.